In [2]:
import numpy as np
import cvxpy as cp
import pandas as pd

df_X = pd.read_csv('Xsvm.csv',header=None)
df_Y = pd.read_csv('ysvm.csv',header=None)
X = np.array(df_X,dtype=np.float64)
Y = np.array(df_Y,dtype=np.float64)
print(X.shape,Y.shape)

(500, 2) (500, 1)


In [52]:
# Convex Optimization
a = cp.Variable(len(Y))
R1 = cp.matmul(cp.diag(a),Y)
R2 = cp.matmul(X.T,R1)
R4 = cp.norm(R2)**2
R4.shape

()

In [53]:
P1 = cp.sum(a)
Const1 = P1 - 0.5*R4
# Const1 = np.reshape(Const1,(1,))

In [ ]:
Const2 = cp.matmul(a.T,Y)
Const3 = [0<=a,Const2 == 0]
obj = cp.Maximize(Const1)
prob = cp.Problem(obj, Const3)
prob.solve(verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2019, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 508, constraints m = 517
Cones:	primal zero / dual free vars: 8
	linear vars: 500
	soc vars: 3, soc blks: 1
	sd vars: 6, sd blks: 1
Setup time: 2.70e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 9.05e+20  1.11e+20  1.00e+00 -2.50e+21 -0.00e+00  1.88e+21  3.16e-02 
   100| 1.61e-01  9.62e-02  1.18e-02 -5.53e+01 -5.66e+01  5.56e-15  6.67e+00 
   200| 7.2

In [32]:
print(a.value)

None
